In [148]:
import pandas as pd
import plotly.express as px
import numpy as np

In [149]:

deaths = pd.read_csv('deaths-drug-overdoses.csv')
dr_all = pd.read_csv('death-rates-from-drug-use-disorders.csv')

In [150]:
dr_all.rename(columns={'Deaths - Drug use disorders - Sex: Both - Age: Age-standardized (Rate)':'deaths'}, inplace=True)
dr_all.head(5)

,Entity,Code,Year,deaths
0,Afghanistan,AFG,1990,1.17
1,Afghanistan,AFG,1991,1.18
2,Afghanistan,AFG,1992,1.21
3,Afghanistan,AFG,1993,1.25
4,Afghanistan,AFG,1994,1.30


In [151]:
dr_all['deaths'].max()

18.83

In [152]:
deaths.head()

,Entity,Code,Year,Deaths - Opioid use disorders - Sex: Both - Age: All Ages (Number),Deaths - Cocaine use disorders - Sex: Both - Age: All Ages (Number),Deaths - Other drug use disorders - Sex: Both - Age: All Ages (Number),Deaths - Amphetamine use disorders - Sex: Both - Age: All Ages (Number)
0,Afghanistan,AFG,1990,73,3,15,2
1,Afghanistan,AFG,1991,80,3,16,2
2,Afghanistan,AFG,1992,93,4,19,3
3,Afghanistan,AFG,1993,104,4,21,3
4,Afghanistan,AFG,1994,112,4,22,3


In [153]:
deaths_colnames = {
    'Deaths - Opioid use disorders - Sex: Both - Age: All Ages (Number)' : "Opioid",
    'Deaths - Cocaine use disorders - Sex: Both - Age: All Ages (Number)' : "Cocaine",
    'Deaths - Other drug use disorders - Sex: Both - Age: All Ages (Number)' : "Other",
    'Deaths - Amphetamine use disorders - Sex: Both - Age: All Ages (Number)' : "Amphetamine"}
deaths.rename(columns=deaths_colnames, inplace=True)


In [154]:
euro_countries = ['Austria','Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia',
                  'Denmark','Estonia','Finland','France','Germany','Greece','Hungary',
                  'Ireland','Italy','Latvia','Lithuania','Luxembourg','Malta','Netherlands',
                  'Poland','Portugal','Romania','Slovakia','Slovenia','Spain','Sweden']

In [155]:
xdeaths = deaths.reset_index()
deaths = pd.melt(xdeaths, id_vars=['Entity', 'Code', 'Year'], value_vars=['Opioid', 'Cocaine', 'Other','Amphetamine'])

In [156]:
deaths = deaths.rename(columns={'variable': 'drug',
                                'value':'total'})
deaths['euro'] = deaths["Entity"].isin(euro_countries) * 1

In [157]:
deaths

,Entity,Code,Year,drug,total,euro
0,Afghanistan,AFG,1990,Opioid,73,0
1,Afghanistan,AFG,1991,Opioid,80,0
2,Afghanistan,AFG,1992,Opioid,93,0
3,Afghanistan,AFG,1993,Opioid,104,0
4,Afghanistan,AFG,1994,Opioid,112,0
...,...,...,...,...,...,...
27355,Zimbabwe,ZWE,2015,Amphetamine,7,0
27356,Zimbabwe,ZWE,2016,Amphetamine,7,0
27357,Zimbabwe,ZWE,2017,Amphetamine,8,0
27358,Zimbabwe,ZWE,2018,Amphetamine,8,0


In [158]:
euro_drugs = deaths[deaths['euro']==1].groupby(['Entity','drug']).sum(['total']).reset_index()
euro_drugs

,Entity,drug,Year,total,euro
0,Austria,Amphetamine,60135,85,30
1,Austria,Cocaine,60135,67,30
2,Austria,Opioid,60135,4327,30
3,Austria,Other,60135,367,30
4,Belgium,Amphetamine,60135,238,30
...,...,...,...,...,...
103,Spain,Other,60135,4129,30
104,Sweden,Amphetamine,60135,607,30
105,Sweden,Cocaine,60135,288,30
106,Sweden,Opioid,60135,5515,30


In [258]:
euro_drugs = deaths[deaths['euro']==1]

In [259]:
t8 = euro_drugs.groupby(['Entity'])['total'].sum().reset_index()
t8 = t8.sort_values('total',ascending=True).tail(8)
#t8.rename(columns={'total' : 'total_entity'}, inplace=True)
#t8['total_8'] = sum(t8['total_entity'])
euro_drugs = euro_drugs[euro_drugs['Entity'].isin(t8['Entity'])]

In [260]:
euro_drugs = euro_drugs.loc[euro_drugs.index.repeat(euro_drugs.total)]

In [261]:
euro_drugs.shape

(151197, 6)

In [186]:
euro_drugs8 = euro_drugs.groupby(['Entity','total_entity','drug'])['total'].sum().reset_index()
euro_drugs8['perc'] = 100 * euro_drugs8['total'] / euro_drugs8.groupby(['Entity'])['total'].transform('sum')
euro_drugs8['perc2'] = 100 * euro_drugs8['total_entity'] / euro_drugs8['total_8']

In [188]:
#euro_drugst = euro_drugs.groupby(['Entity','total_entity','drug'])['total'].sum().reset_index()
#euro_drugst['perc'] = 100 * euro_drugst['total'] / euro_drugst.groupby(['Entity'])['total'].transform('sum')
#euro_drugs['perc2'] = 100 * euro_drugs['total'] / euro_drugs.groupby(['Entity'])['total'].transform('sum')

In [262]:

fig_drugs = px.sunburst(euro_drugs,
                        path = ['Entity','drug'],
                        #values = 'perc',
                        color = 'drug',
                        color_discrete_sequence = px.colors.sequential.Greens).update_traces(hovertemplate = '%{label}<br>' + 'Global Drugs: %{value}%', textinfo = "label + percent entry") 

fig_drugs = fig_drugs.update_layout({'margin' : dict(t=0, l=0, r=0, b=10),
                        'paper_bgcolor': '#F9F9F8',
                        'font_color':'#9dc943'})

fig_drugs

c:\Users\Crs\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\Crs\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [206]:
import pandas as pd
import plotly.express as px


data = {
  'ids':['SA', 'NA', 'Brazil', 'Uruguay', 'USA', 'Canada', 'PFV Brazil', 'PV Brazil', 'PFV Uruguay', 'PV Uruguay', 'PFV USA', 'PV USA', 'PFV Canada', 'PV Canada'],
  'labels': ['SA', 'NA', 'Brazil', 'Uruguay', 'USA', 'Canada', 'PFV', 'PV', 'PFV', 'PV', 'PFV', 'PV', 'PFV', 'PV'],
  'parent': ['', '', 'SA', 'SA', 'NA', 'NA', 'Brazil', 'Brazil', 'Uruguay', 'Uruguay', 'USA', 'USA', 'Canada', 'Canada'],
  'value': [0, 0, 100, 80, 400, 200, 8, 40, 4, 20, 11, 100, 11, 80]
  }

fig =px.sunburst(data, names='labels', parents='parent',  values='value', ids='ids')
fig.show()

In [17]:
death_sub = deaths.groupby(['drug'])['total'].sum().reset_index()
death_sub = death_sub.sort_values('total',ascending=True)

In [49]:
death_sub = deaths[deaths['drug'] == "Opioid"]
death_sub = death_sub.groupby(['Entity'])['total'].sum().reset_index()
death_sub = death_sub.sort_values('total',ascending=True).tail(8)

In [16]:
death_sub

,drug,total
1,Cocaine,1212607
0,Amphetamine,1404294
3,Other,2550514
2,Opioid,11583657


In [34]:
fig = px.bar(death_sub,
       x="total",
       y='Entity',
       orientation='h')

fig.show()

In [6]:
focus = pd.read_csv("./focus.csv")
focus['text'] = focus['entity']+ '<br>' + focus['focus']

In [29]:
import plotly.graph_objs as go
data_slider = []
data_each_yr = dict(type='choropleth',
                        locations = focus['entity'],
                        locationmode='country names',                        
                        autocolorscale = False,
                        z=focus['focus2'],                                              
                        colorscale = ["#ffe2bd", "#006837"],
                        #showscale=True,
                        marker_line_color='#006837',
                        text=focus['text'],                                               
                        colorbar=dict(title='', 
                                      tickvals=[1, 2], 
                                      ticktext=['Broader focus', 'Illicit drugs focus'],
                                      tickmode='array',
                                      thicknessmode='fraction',
                                      ticklen=1,
                                      tickcolor='#000'),

                    )
data_slider.append(data_each_yr)
layout = dict(geo=dict(scope='europe',
                            projection={'type': 'natural earth'},
                            showlakes=True,
                            lakecolor='rgb(255, 255, 255)',
                            bgcolor= 'rgba(0,0,0,0)'),
                    margin=dict(l=0,
                                r=0,
                                b=0,
                                t=30,
                                pad=0),
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    dragmode='pan', # disable zoom
                    )

 
fig_choropleth = go.Figure(data=data_slider, layout=layout)
fig_choropleth.update_geos(showcoastlines=False, showsubunits=True,showframe=True)
#fig_choropleth.update_xaxes(fixedrange=True)


In [268]:
layout = dict(geo=dict(scope='europe',
                            projection={'type': 'natural earth'},
                            showlakes=True,
                            lakecolor='rgb(255, 255, 255)',
                            bgcolor= 'rgba(0,0,0,0)'),
                    margin=dict(l=0,
                                r=0,
                                b=0,
                                t=30,
                                pad=0),
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)')
 
fig_choropleth = go.Figure(data=data_slider, layout=layout)
fig_choropleth.update_geos(showcoastlines=False, showsubunits=True,showframe=True)


ValueError: Invalid property specified for object of type plotly.graph_objs.Choropleth: 'legend'

Did you mean "geo"?

    Valid properties:
        autocolorscale
            Determines whether the colorscale is a default palette
            (`autocolorscale: true`) or the palette determined by
            `colorscale`. In case `colorscale` is unspecified or
            `autocolorscale` is true, the default  palette will be
            chosen according to whether numbers in the `color`
            array are all positive, all negative or mixed.
        coloraxis
            Sets a reference to a shared color axis. References to
            these shared color axes are "coloraxis", "coloraxis2",
            "coloraxis3", etc. Settings for these shared color axes
            are set in the layout, under `layout.coloraxis`,
            `layout.coloraxis2`, etc. Note that multiple color
            scales can be linked to the same color axis.
        colorbar
            :class:`plotly.grap

In [32]:
gkk = deaths[deaths['euro']==1].groupby(['euro', 'Entity','drug'])['total'].sum().reset_index()
gkk['perc'] = 100 * gkk['total'] / gkk.groupby(['euro', 'Entity'])['total'].transform('sum')
gkk

,euro,Entity,drug,total,perc
0,1,Austria,Amphetamine,85,1.754024
1,1,Austria,Cocaine,67,1.382584
2,1,Austria,Opioid,4327,89.290136
3,1,Austria,Other,367,7.573256
4,1,Belgium,Amphetamine,238,4.817814
...,...,...,...,...,...
103,1,Spain,Other,4129,17.480948
104,1,Sweden,Amphetamine,607,7.878001
105,1,Sweden,Cocaine,288,3.737833
106,1,Sweden,Opioid,5515,71.576898


In [50]:
country = ['Afghanistan']

In [58]:
fig2 = px.line(deaths[deaths["Entity"].isin(country)], x = "Year", y='total', color='drug')  

In [59]:
fig2 = fig2.update_layout({'margin': dict(t=0, l=0, r=0, b=0),
                               'paper_bgcolor': '#F9F9F8',
                               'font_color': '#363535'})

In [60]:
fig2

In [17]:
x=deaths['Deaths - Opioid use disorders - Sex: Both - Age: All Ages (Number)']
x

0        73
1        80
2        93
3       104
4       112
       ... 
6835     77
6836     81
6837     85
6838     88
6839     92
Name: Deaths - Opioid use disorders - Sex: Both - Age: All Ages (Number), Length: 6840, dtype: int64